In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

df = pd.read_csv(r"C:\ise3230\nutrition_and_price_dataset.csv")

items = df["food_name"].tolist()

fat = df.set_index("food_name")["fat_g"].to_dict()
sodium = df.set_index("food_name")["sodium_mg"].to_dict()
protein = df.set_index("food_name")["protein_g"].to_dict()
calories = df.set_index("food_name")["calories_kcal"].to_dict()
cost = df.set_index("food_name")["price_per_serving_usd"].to_dict()

m = gp.Model("diet")
m.Params.LogToConsole = 0

x = m.addVars(items, vtype=GRB.INTEGER, lb=0, name="x")

m.setObjective(gp.quicksum(cost[i] * x[i] for i in items), GRB.MINIMIZE)

FAT_MAX = 275
SODIUM_MAX = 4000
PROTEIN_MIN = 150
CALORIE_MIN = 10000

m.addConstr(gp.quicksum(fat[i] * x[i] for i in items) <= FAT_MAX, "fat_max")
m.addConstr(gp.quicksum(sodium[i] * x[i] for i in items) <= SODIUM_MAX, "sodium_max")
m.addConstr(gp.quicksum(protein[i] *  x[i] for i in items) >= PROTEIN_MIN, "protein_min")
m.addConstr(gp.quicksum(calories[i] * x[i] for i in items) >= CALORIE_MIN, "calorie_max")

m.optimize()

if m.Status == GRB.OPTIMAL:
    print("obj =", m.ObjVal)
    for i in items:
        if x[i].X > 1e-6:
            print(i, int(round(x[i].X)))
else:
    print("Status:", m.Status)

Set parameter LogToConsole to value 0
obj = 13.63
Instant oatmeal packet 38
Bagel (plain) 1
Pickles 7
